In [1]:
%run uvms_base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# conditional function instances
# coupled forward dynamics of uvms
x_next_coupled, x_nextdd_coupled, states, u, dt, q_min, q_max, manipulator_p, vehicle_p, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=True)
x_next_coupled_eval = cs.Function('UVMSnext_coupled', [states, u, dt, vehicle_p, manipulator_p, base_T ,q_min, q_max], [x_next_coupled, u_checks, x_nextdd_coupled])


# decoupled forward dynamics of uvms
x_next_decoupled, x_nextdd_decoupled, states, u, dt, q_min, q_max, manipulator_p, vehicle_p, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=False)

x_next_decoupled_eval = cs.Function('UVMSnext_decoupled', [states, u, dt, vehicle_p, manipulator_p, base_T ,q_min, q_max], [x_next_decoupled, u_checks, x_nextdd_decoupled])



UVMSnext_use_coupled = cs.Function.if_else('UVMSnext_use_coupled', x_next_coupled_eval, x_next_decoupled_eval)
UVMSnext_use_coupled.save('uvms_forward_dynamics.casadi')

floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found


In [3]:
manipulator_p

SX([G_0, G_1, G_2, G_3, Ir3_14, Ir2_7, Ir1_7, Ir0_14, fw_static_0, fw_static_1, fw_static_2, fw_static_3, fw_viscous_0, fw_viscous_1, fw_viscous_2, fw_viscous_3, bw_static_0, bw_static_1, bw_static_2, bw_static_3, bw_viscous_0, bw_viscous_1, bw_viscous_2, bw_viscous_3])

In [3]:
# # c , cpp or matlab code generation for forward dynamics
# UVMSnext_use_coupled.generate("x_next_eval.c")
# os.system(f"gcc -fPIC -shared x_next_eval.c -o libUVMS_xnext.so")

0

In [5]:
st_x0 = cs.vertcat(0,0,0, 0,0,0, 0.0, 3.4, 0.99245, 3.16073,   0, 0, 0, 0, 0, 0,  0.00751799, -0.0073626, 0.00281749, 0.0222385)
u0 = cs.vertcat(0.0, 0, 0, 0, 0, 0, 0.0, 1.0, 0.0, 0.0)

UVMSnext_use_coupled(use_coupled, st_x0, u0, alpha.delta_t, blue.sim_p, alpha.sim_p, alpha.base_T0, alpha.joint_min, alpha.joint_max)

(DM([-8.74096e-07, 7.0552e-07, 1.45002e-05, -1.23881e-05, 2.56853e-05, -3.20227e-05, 5.6418e-05, 3.39991, 0.992479, 3.16098, -2.30097e-05, 1.75013e-05, -0.00158545, -0.00030136, 0.000654206, -0.000767976, 9.71974e-06, -0.000135707, 2.13643e-05, 0.000271332]),
 DM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 DM([-0.0081161, 0.00300693, -0.00298098, 0.00342372, 0.143811, -0.01651, -0.0497736, 2.24825, -0.0137912, -0.0979682]))